In [1]:
import os
import json
from typing import Optional, Tuple
import numpy as np

import pyproj

In [2]:
meataDta_file = "C:/Users/admin/Desktop/codespace/python/output_sfm/exif_data.json"

In [3]:
if os.path.isfile(meataDta_file) is not True:
    raise NotADirectoryError(f"The File {meataDta_file} not found")

metadata = []
with open(meataDta_file) as f:
    metadata = json.load(f)
print(len(metadata))
print(metadata[0])

19
{'file': 'IMG_20200328_172713', 'image_id': 'IMAGE001', 'exif_version': '0220', 'make': 'Xiaomi', 'model': 'Redmi 4', 'width': 2340, 'height': 4160, 'orientation': 1, 'capture_time': 1585416433.065576, 'altitude': 0.0, 'altitude_ref': 2.0, 'latitude': [22.0, 34.0, 8.8892], 'latitude_ref': 'N', 'longitude': [88.0, 17.0, 0.038], 'longitude_ref': 'E', 'dop': None}


In [4]:
def decimal_coords(coords: Tuple[float, float, float], ref: Optional[str]) -> float:
        """Convert Latitude Longitude to Degree."""
        decimal_degrees = coords[0] + coords[1] / 60.0 + coords[2] / 3600.0
        if ref == "S" or ref == "W":
            decimal_degrees = -decimal_degrees
        return decimal_degrees

In [5]:
obj=metadata[0]
latitude = decimal_coords(obj.get("latitude"),obj.get("latitude_ref"))
print(f'latitude {obj.get("latitude")} {obj.get("latitude_ref")} ==> {latitude}')
obj=metadata[0]
longitude = decimal_coords(obj.get("longitude"),obj.get("longitude_ref"))
print(f'longitude {obj.get("longitude")} {obj.get("longitude_ref")} ==> {longitude}')
altitude=float(obj.get("altitude"))
if float(obj.get("altitude_ref")) == 1:
    altitude = -altitude
print(f'altitude {obj.get("altitude")} {obj.get("altitude_ref")} ==> {altitude}')    


latitude [22.0, 34.0, 8.8892] N ==> 22.569135888888887
longitude [88.0, 17.0, 0.038] E ==> 88.2833438888889
altitude 0.0 2.0 ==> 0.0


In [6]:
WGS84_a = 6378137.0
WGS84_b = 6356752.314245


def ecef_from_lla(lat, lon, alt):
    """
    Compute ECEF XYZ from latitude, longitude and altitude.
    All using the WGS84 model.
    Altitude is the distance to the WGS84 ellipsoid.
    Check results here http://www.oc.nps.edu/oc2902w/coord/llhxyz.htm
    >>> lat, lon, alt = 10, 20, 30
    >>> x, y, z = ecef_from_lla(lat, lon, alt)
    >>> np.allclose(lla_from_ecef(x,y,z), [lat, lon, alt])
    True
    """
    a2 = WGS84_a ** 2
    b2 = WGS84_b ** 2
    lat = np.radians(lat)
    lon = np.radians(lon)
    L = 1.0 / np.sqrt(a2 * np.cos(lat) ** 2 + b2 * np.sin(lat) ** 2)
    x = (a2 * L + alt) * np.cos(lat) * np.cos(lon)
    y = (a2 * L + alt) * np.cos(lat) * np.sin(lon)
    z = (b2 * L + alt) * np.sin(lat)
    return x, y, z

def ecef_from_topocentric_transform(lat, lon, alt):
    """
    Transformation from a topocentric frame at reference position to ECEF.
    The topocentric reference frame is a metric one with the origin
    at the given (lat, lon, alt) position, with the X axis heading east,
    the Y axis heading north and the Z axis vertical to the ellipsoid.
    >>> a = ecef_from_topocentric_transform(30, 20, 10)
    >>> b = ecef_from_topocentric_transform_finite_diff(30, 20, 10)
    >>> np.allclose(a, b)
    True
    """
    x, y, z = ecef_from_lla(lat, lon, alt)
    sa = np.sin(np.radians(lat))
    ca = np.cos(np.radians(lat))
    so = np.sin(np.radians(lon))
    co = np.cos(np.radians(lon))
    return np.array(
        [
            [-so, -sa * co, ca * co, x],
            [co, -sa * so, ca * so, y],
            [0, ca, sa, z],
            [0, 0, 0, 1],
        ]
    )

def topocentric_from_lla(lat, lon, alt, reflat, reflon, refalt):
    """
    Transform from lat, lon, alt to topocentric XYZ.
    >>> lat, lon, alt = -10, 20, 100
    >>> np.allclose(topocentric_from_lla(lat, lon, alt, lat, lon, alt),
    ...     [0,0,0])
    True
    >>> x, y, z = topocentric_from_lla(lat, lon, alt, 0, 0, 0)
    >>> np.allclose(lla_from_topocentric(x, y, z, 0, 0, 0),
    ...     [lat, lon, alt])
    True
    """
    T = np.linalg.inv(ecef_from_topocentric_transform(reflat, reflon, refalt))
    x, y, z = ecef_from_lla(lat, lon, alt)
    tx = T[0, 0] * x + T[0, 1] * y + T[0, 2] * z + T[0, 3]
    ty = T[1, 0] * x + T[1, 1] * y + T[1, 2] * z + T[1, 3]
    tz = T[2, 0] * x + T[2, 1] * y + T[2, 2] * z + T[2, 3]
    return tx, ty, tz

In [7]:
CECF=ecef_from_lla(latitude,longitude,altitude)
print(f"{CECF}<=ecef_from_lla({latitude},{longitude},{altitude})")

(176523.15429462085, 5889942.134759544, 2432728.374973132)<=ecef_from_lla(22.569135888888887,88.2833438888889,0.0)


In [15]:
transformer1 = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )
x = 176523.15429462085
y = 5889942.134759544
z = 2432728.374973132
lon1, lat1, alt1 = transformer1.transform(x,y,z,radians=False)
print(lon1, lat1, alt1)

transformer2 = pyproj.Transformer.from_crs(
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    )
x,y,z = transformer2.transform(longitude,latitude,altitude,radians=False)
print(x,y,z)


88.2833438888889 22.569135888887743 -2.60770320892334e-08
176523.15429462015 5889942.13475952 2432728.3749732594


ProjError: x, y, z, and time must be same size if included.